<a href="https://colab.research.google.com/github/isabelladegen/ComputationalLogic/blob/prolexa-plus/Demo_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Demo and Testing Notebook Isabella
Notebook that makes sure Prolexa continues to work


### Installations like demo notebook


In [1]:
!apt-get install swi-prolog -qqq > /dev/null

Extracting templates from packages: 100%


### Installing Prolexa

Using --upgrade to always pull changes from the git repostiory!
*Same error as on the branch on the forked branch https://github.com/simply-logical/ComputationalLogic/*

In [2]:
!yes | pip install --upgrade git+https://github.com/isabelladegen/ComputationalLogic/ -qqq > /dev/null

  ERROR: Failed building wheel for prolexa
  DEPRECATION: prolexa was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


### Functions to call Prolexa directly

These are not using the meta_grammar from Prolexa, meaning that no know proper nouns can be added


In [3]:
import os
from pyswip import Prolog
import site
 
def find_prolog_path():
  packages_paths = site.getsitepackages()
  for ppath in packages_paths:
    if 'prolexa' in os.listdir(ppath):
      return os.path.join(ppath, 'prolexa/prolog')
    else:
      raise Exception(f'\'prolexa\' package was not found in {packages_path}. Did you install it?')
                
pl = Prolog()

def initialise_prolexa():
  pl.consult(os.path.join(find_prolog_path(), 'prolexa.pl'))

def query(input): #also inspired from prolexa
  initialise_prolexa()
  libPrefix = 'prolexa:'
  generator = pl.query(libPrefix + handle_utterance_str(input))
  answer = list(generator)[0]['Output']
  if isinstance(answer, str):
    return answer
  return str(answer, 'utf-8')

def handle_utterance_str(text): #this is from the meta_grammar from prolexa plus!
    if text[0] != "'" and text[0] != '"' :
        text = f'"{text}"'

    text = text.replace('"', '\"')
    text = text.replace("'", '\"')

    return 'handle_utterance(1,{},Output)'.format(text)

In [4]:
query('Tell me all')

'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird'

###  Some Helper functions to make tests easier to read

In [5]:
def test(input, expected_answer):
  # print(f'Query: {input}')
  actual_answer = query(input)
  # print(f'Answer: {actual_answer}\n')
  assert actual_answer == expected_answer, 'Expected Answer:  ' + expected_answer + '\n Actual answer: ' + actual_answer

def cleanup():
  query('forget everything')
  initialise_prolexa()


### Ensure existing functionality continues to work

Syntax: `test(input_query, expected_answer)`


In [37]:
cleanup() #celean, in case some tests added knowledge
#Check whats known in the stored rules
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')

#Tell me about... queries
test('Tell me all', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird') #extened with new rules added to prolexa
test('tell me about unknownnoun', 'I heard you say,  tell me about unknownnoun , could you rephrase that please?')
test('tell me about tweety', 'I know nothing about tweety')
test('tell me about peter', 'peter is human. peter is mortal')
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')

#Is... queries
test('Is unkownnoun human', 'I heard you say,  Is unkownnoun human , could you rephrase that please?')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Is peter mortal', 'peter is mortal')
test('Is peter human', 'peter is human')
test('Who is human', 'peter is human')
test('Who is a bird', 'opus is a bird')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')

#Explain... queries
test('Explain why unknownnoun is mortal', 'I heard you say,  Explain why unknownnoun is mortal , could you rephrase that please?') #unkown words don't work
test('Explain why tweety is a bird','Sorry, I don\'t think this is the case')
test('Explain why peter is mortal', 'peter is human; every human is mortal; therefore peter is mortal')

#Adding new rules and forgeting rules
test('Peter is mortal', 'I already knew that Peter is mortal')
test('Tweety is a bird', 'I will remember that Tweety is a bird')
test('Is tweety a bird', 'tweety is a bird')
test('Tell me all about tweety', 'tweety is a bird. tweety flies') 
test('Forget that tweety is a bird', 'I erased it from my memory')
test('tell me all about tweety', 'I know nothing about tweety')
test('Is tweety a bird', 'Sorry, I don\'t think this is the case')
test('Does tweety fly', 'Sorry, I don\'t think this is the case')
cleanup()


# Demonstrating new functionality


### Default Reasoning with rules in the rulebase
- Peep and Opus both are proper nouns
- <TODO: put the rulebase here>

In [45]:
cleanup()
#Check what's already known about tweety, peep and opus in the rulebase before we start adding
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')

#Queries
test('Tell me about opus', 'opus is a bird. opus is a penguin. opus doesnt fly') #reports on negated rules from other truths: not(fly(X):-penguin(X)) & penguin(opus):-true
test('Tell me about peep', 'peep is a bird. peep flies')
test('Is opus a bird', 'opus is a bird')
test('Is peep a bird', 'peep is a bird')
test('Is opus a penguin', 'opus is a penguin')
test('Is peep a penguin', 'Sorry, I don\'t think this is the case')
test('Does opus fly', 'Sorry, I don\'t think this is the case')
test('Does peep fly','peep flies')

#Explanations
test('Explain why peep flies', 'peep is a bird; some birds fly; therefore peep flies') #explaining using default rules for flying
test('Explain why opus flies', 'Sorry, I don\'t think this is the case') #cannot explain that opus flies (cause opus doesn't fly)
test('Explain why opus doesnt fly', 'opus is a penguin; penguins dont fly; therefore opus doesnt fly') #can explain that opus doesn't fly

#Check what's known now
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')
cleanup()


### Default reasoning creating new rules while talking to prolexa
- the vocabulary has the verbs sing and fly and the noun swan added, there are no rules about swans and sining in the rulebase
- Some... will be interpreted as a default rule with exception stored_rule[default(H:B)]
- All... is interpreted as a normal stored rule: stored_rule[H:-B]

In [36]:
cleanup()
#Check what's already known about tweety, peep and opus in the rulebase before we start adding
test('Spill the beans', 'every human is mortal. peter is human. some birds fly. penguins dont fly. every penguin is a bird. opus is a penguin. peep is a bird')
test('Tell me about tweety', 'I know nothing about tweety')
test('Tell me about peep', 'peep is a bird. peep flies')
test('Tell me about opus', 'opus is a bird. opus is a penguin. opus doesnt fly')

#Create new Default and normal rules
test('Some birds sing', 'I will remember that Some birds sing') #adds a new default: stored_rule([(default(bird(X):-sing(X)))])
test('Swans dont sing', 'I will remember that Swans dont sing') #adds a new rule: stored_rule([(not sing(X):-swan(X))])
test('All swans are birds', 'I will remember that All swans are birds') #adds a new rule: stored_rule([(bird(X):-swan(X))])
test('Tweety is a swan', 'I will remember that Tweety is a swan') #adds a new rule: stored_rule([(swan(tweety):-true)])

#Show what is known about all the different birds with the new rules
test('Tell me about tweety', 'tweety is a bird. tweety is a swan. tweety flies. tweety doesnt sing')
test('Tell me about peep', 'peep is a bird. peep flies. peep sings')
test('Tell me about opus', 'opus is a bird. opus is a penguin. opus doesnt fly. opus sings')

#Demonstrate explanations
test('Explain why peep sings', 'peep is a bird; some birds sing; therefore peep sings')
test('Explain why tweety doesnt sing', 'tweety is a swan; swans dont sing; therefore tweety doesnt sing') 
test('Explain why tweety flies', 'tweety is a swan; every swan is a bird; some birds fly; therefore tweety flies') 
test('Explain why opus sings', 'opus is a penguin; every penguin is a bird; some birds sing; therefore opus sings')

#Demonstrate questions
test('Does tweety sing', 'Sorry, I don\'t think this is the case')
test('Does opus sing', 'opus sings')
test('Does peep sing', 'peep sings')

test('Does tweety fly', 'tweety flies')
test('Does opus fly', 'Sorry, I don\'t think this is the case')
test('Does peep fly', 'peep flies')

test('Is tweety a bird', 'tweety is a bird')
test('Is opus a bird', 'opus is a bird')
test('Is peep a bird', 'peep is a bird')

# Negation
This has been indirectly implemented as part of default reasoning

In [34]:
cleanup()
#check what's already in the rulebase
test('Tell me about Peter', 'peter is human. peter is mortal')

#Create a negated fact about Peter
test('Peter doesnt fly', 'I will remember that Peter doesnt fly') #adds a new rule: stored_rule([(not fly(peter):-true)])

#check what's now known about Peter
test('Tell me about Peter', 'peter is human. peter is mortal. peter doesnt fly')
test('Explain why Peter doesnt fly', 'peter doesnt fly; therefore peter doesnt fly') 